In [60]:
import dryscrape
import time

# Programa para bajar cosas

- uno
- dos
- y después viene el tres


In [78]:
sess = dryscrape.Session()

url = 'https://www.wikiart.org/es'

sess.visit(url)

loadmore_button = sess.xpath("//*[@class = 'loadmore-icon']")[0]

loadmore_button.click()
time.sleep(2)

img_src = sess.xpath("//*[@class = 'artwork-name ng-binding']")

In [79]:


img_src[0].get_attr('href')

u'/es/theo-van-rysselberghe/the-mill-at-kelf-1894'

In [64]:
loadmore_button.click()
time.sleep(2)

img_src = sess.xpath("//*[@class = 'artwork-name ng-binding']/@href")


print(len(img_src))



320


In [73]:
import numpy as np


In [72]:
np.

dryscrape.driver.webkit.Node

In [19]:
sess = dryscrape.Session()

sess.visit('https://www.wikiart.org/es/ende/beato-de-gerona-975')

In [33]:
a = sess.xpath('//*[@class = "ms-zoom-cursor"]')[0]

In [41]:
img_src = a['src']
img_title = a['title']

In [42]:
import requests
import shutil

response = requests.get(img_src, stream=True)
with open('{}.jpg'.format(img_title), 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
del response

In [40]:
a['title']

u'Beato de Gerona, c.975 - Ende'